https://github.com/chiarasivieri/ArchivIA

In [ ]:
!pip install piexif -q

In [ ]:
!python --version

Python 3.12.11


In [ ]:
import os
import csv
import logging
import argparse
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

from tqdm import tqdm
from PIL import Image, UnidentifiedImageError
import piexif
import shutil
from types import SimpleNamespace

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### **Configurazione dei Percorsi**

* **GDRIVE_PROJECT_PATH**= Path(...): Definisce il percorso principale della
cartella del progetto su *Google Drive.* Utilizza la libreria pathlib.
* **DEFAULT_TRAIN_DIR** = GDRIVE_PROJECT_PATH / "dataset" / "train": A partire dal percorso base, costruisce i percorsi specifici per i dati. L'operatore / che funziona correttamente su qualsiasi sistema operativo (Windows, Mac, Linux).
* **DEFAULT_TRAIN_DIR:** Cartella con le immagini per l'addestramento.
* **DEFAULT_VAL_DIR:** Cartella con le immagini per la validazione (usata durante l'addestramento per monitorare le performance).
* **DEFAULT_TEST_DIR:** Cartella con le immagini per il test finale (usata dopo l'addestramento per una valutazione imparziale).
* **DEFAULT_IMG_DIR e DEFAULT_VID_DIR:** Cartelle da cui prendere immagini e video per l'inferenza (l'uso pratico del modello). In questo caso, puntano alla cartella test.
* **DEFAULT_OUTPUT:** La cartella dove verranno salvati tutti i risultati (modelli, log, CSV, immagini classificate).

* **DEFAULT_OUTPUT.mkdir(...):** Questo comando crea fisicamente le cartelle di output su Drive. Argomenti:
1. **parents=True:** Se la cartella genitore non esiste (es. PROGETTO), la crea.
2. **exist_ok=True:** Se la cartella esiste già, non genera un errore.


In [ ]:
# Configurazione Percorso Base del Progetto - da Drive
GDRIVE_PROJECT_PATH = Path("/content/drive/MyDrive/PROGETTO")
print(f"Percorso base del progetto impostato a: {GDRIVE_PROJECT_PATH}")


# CONFIGURAZIONE PARAMETRI GLOBALI
# (Questi non cambiano)
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS_HEAD = 10
EPOCHS_FINE = 8
FRAME_INTERVAL = 1 # L'intervallo in secondi tra i fotogrammi estratti da un video.
LEARN_HEAD_LR = 1e-3 #approccio a step discreti. Si inizia con un learning rate (LR) alto e costante per N epoche.
# Poi, si stoppa, si cambia manualmente il LR a un valore molto più basso, e si riparte per M epoche
LEARN_FULL_LR = 1e-5
SHARPNESS_THRESHOLD = 80.0 # La soglia minima di "nitidezza" che un frame deve avere per essere considerato valido.
FRAMES_PER_PLACE_PER_VIDEO = 5


#PERCORSI DI DEFAULT DERIVATI (DA MODIFICARE)

# Percorsi per Training, Validazione e Test
DEFAULT_TRAIN_DIR = GDRIVE_PROJECT_PATH / "dataset" / "train"
DEFAULT_VAL_DIR = GDRIVE_PROJECT_PATH / "dataset" / "val"
DEFAULT_TEST_DIR = GDRIVE_PROJECT_PATH / "dataset" / "test"

# Percorsi per l'Inferenza (puntano alla cartella 'test')
# In questo modo, quando si esegue l'inferenza, userà le immagini e i video di test.
DEFAULT_IMG_DIR = GDRIVE_PROJECT_PATH / "dataset" / "test"
DEFAULT_VID_DIR = GDRIVE_PROJECT_PATH / "dataset" / "test"

# Cartella principale dove verranno salvati tutti gli output
DEFAULT_OUTPUT = GDRIVE_PROJECT_PATH / "output"

# Percorsi derivati per l'output
DEFAULT_CSV = DEFAULT_OUTPUT / "results.csv"
PROCESSED_LOG = DEFAULT_OUTPUT / "processed.log"
BEST_FRAMES_OUTPUT_DIR = DEFAULT_OUTPUT / "best_frames"
TEMP_FRAME_EXTRACT_DIR = DEFAULT_OUTPUT / "temp_frames"


#Creazione delle Directory di Output
DEFAULT_OUTPUT.mkdir(parents=True, exist_ok=True)
BEST_FRAMES_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
TEMP_FRAME_EXTRACT_DIR.mkdir(parents=True, exist_ok=True)


# Stampa di Verifica
# Aggiornata per includere il nuovo percorso di test
print(f"\n Percorsi Configurati ")
print(f"Training Data: {DEFAULT_TRAIN_DIR}")
print(f"Validation Data: {DEFAULT_VAL_DIR}")
print(f"Test Data: {DEFAULT_TEST_DIR}")
print(f"Input Immagini/Video (Inferenza): {DEFAULT_IMG_DIR}")
print(f"Output Generale: {DEFAULT_OUTPUT}")

Percorso base del progetto impostato a: /content/drive/MyDrive/PROGETTO

 Percorsi Configurati 
Training Data: /content/drive/MyDrive/PROGETTO/dataset/train
Validation Data: /content/drive/MyDrive/PROGETTO/dataset/val
Test Data: /content/drive/MyDrive/PROGETTO/dataset/test
Input Immagini/Video (Inferenza): /content/drive/MyDrive/PROGETTO/dataset/test
Output Generale: /content/drive/MyDrive/PROGETTO/output


### **Preparazione Dataset + data aug**

* `make_dataset`: Trasforma una cartella di immagini in un `tf.data.Dataset`, un oggetto TensorFlow ottimizzato per l'addestramento.

*   **`image_dataset_from_directory`**:
    * (in teoria) Scansiona la struttura delle directory (es. `/train/economia_interno/img1.jpg`) e inferisce automaticamente che `img1.jpg` appartiene alla classe `economia_interno`.
    *   **`label_mode="categorical"`**: È una scelta cruciale. Trasforma le etichette testuali in vettori "one-hot" (es. per due classi, `economia_interno` diventa `[1, 0]` e `stum_interno` diventa `[0, 1]`). Questo è il formato matematico richiesto dalla funzione di perdita `categorical_crossentropy` per funzionare correttamente.

*   **Catena di Pre-processing con `.map()`**:
    * Invece di processare le immagini una per una in un ciclo `for`, il metodo `.map()` applica una funzione a interi batch di dati in modo parallelizzato.
    *   **`preprocess_input`**: Ogni modello pre-addestrato, si aspetta che i valori dei pixel delle immagini siano normalizzati in un modo molto specifico (tipo scalati in un range da -1 a 1). La funzione `preprocess_input` fornita da Keras esegue esattamente questa normalizzazione, garantendo che i dati che il nostro modello vede siano nello stesso formato dei dati su cui è stato originariamente addestrato (ImageNet).

    *   **`prefetch(tf.data.AUTOTUNE)`**: Permette alla pipeline di dati di lavorare in modo asincrono: mentre la GPU sta addestrando sul batch `N`, la CPU sta già preparando il batch `N+1` in background. `AUTOTUNE` lascia che sia TensorFlow a decidere dinamicamente quanti batch pre-caricare per ottenere le massime prestazioni. (in teoria ottimizza un po' il tutto)

In [ ]:
# Pipeline di Data Augmentation
# La Data Augmentation crea nuove immagini di training al volo, applicando trasformazioni casuali.
# Questo aiuta a prevenire l'overfitting e rende il modello più robusto a variazioni nelle immagini reali.
# `tf.keras.Sequential` crea una pipeline dove i dati passano attraverso i layer in sequenza.
data_augmentation_pipeline = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),        # Specchia l'immagine orizzontalmente con una probabilità del 50%.
    layers.RandomRotation(0.15),            # Ruota l'immagine di un angolo casuale fino al 15% di 360 gradi.
    layers.RandomZoom(0.15),                # Ingrandisce o rimpicciolisce l'immagine fino al 15%.
    layers.RandomContrast(0.1),             # Modifica il contrasto in modo casuale.
    layers.RandomBrightness(0.1),           # Modifica la luminosità in modo casuale.
], name="data_augmentation")


# Funzione per creare i dataset
# Questa funzione carica le immagini da una directory, le pre-processa e le prepara in un formato
# efficiente per l'addestramento con TensorFlow (`tf.data.Dataset`).
def make_dataset(dirpath: Path, shuffle: bool, subset: str = None, validation_split: float = None, augment: bool = False):
    # Controllo preliminare per assicurarsi che la directory esista.
    if not dirpath.is_dir():
        logging.error(f"MAKE_DATASET: Directory non trovata: {dirpath}")
        return None, None # Restituisce None per segnalare l'errore.
    try:
        # Funzione di utility di Keras che fa gran parte del lavoro pesante.
        # Scansiona la directory, inferisce i nomi delle classi dalle sottocartelle,
        # e carica le immagini.
        initial_ds = tf.keras.utils.image_dataset_from_directory(
            dirpath,                      # Il percorso della cartella da cui caricare le immagini.
            labels="inferred",            # I nomi delle classi sono dedotti dai nomi delle sottocartelle.
            label_mode="categorical",     # Le etichette sono convertite in formato one-hot (es. [0, 1, 0]).
            batch_size=BATCH_SIZE,        # Raggruppa le immagini in batch.
            image_size=IMG_SIZE,          # Ridimensiona tutte le immagini a IMG_SIZE.
            shuffle=shuffle,              # Mescola i dati (importante per il training).
            seed=123,                     # Seed per la riproducibilità del mescolamento e dello split.
            validation_split=validation_split, # Frazione di dati da riservare per la validazione.
            subset=subset,                # Specifica se creare il set di 'training' o 'validation'.
            interpolation='bilinear'      # Algoritmo usato per il ridimensionamento delle immagini.
        )
        # Estrae i nomi delle classi trovati dalla funzione.
        class_names = initial_ds.class_names

        # Applica la data augmentation solo se richiesto (tipicamente solo per il set di training).
        processed_ds = initial_ds
        if augment:
            # `map` applica una funzione a ogni elemento (batch) del dataset.
            # `lambda imgs, labs:` è una funzione anonima che prende un batch di immagini e le loro etichette.
            # Applica la pipeline di augmentation solo alle immagini, lasciando le etichette invariate.
            # `num_parallel_calls=tf.data.AUTOTUNE` permette a TensorFlow di parallelizzare l'operazione per la massima efficienza.
            processed_ds = processed_ds.map(lambda imgs, labs: (data_augmentation_pipeline(imgs, training=True), labs),
                                           num_parallel_calls=tf.data.AUTOTUNE)

        # Applica la funzione di preprocessing specifica del modello (es. scalare i pixel da [0, 255] a [-1, 1]).
        # Questo passo è OBBLIGATORIO e va fatto sia per il training che per la validazione/inferenza.
        processed_ds = processed_ds.map(lambda imgs, labs: (preprocess_input(imgs), labs),
                                       num_parallel_calls=tf.data.AUTOTUNE)

        # `prefetch(tf.data.AUTOTUNE)` è un'ottimizzazione delle performance.
        # Permette alla CPU di preparare il batch successivo di dati mentre la GPU sta elaborando quello corrente.
        # Questo previene i "colli di bottiglia" dovuti al caricamento dei dati.
        return processed_ds.prefetch(tf.data.AUTOTUNE), class_names
    except Exception as e:
        logging.error(f"MAKE_DATASET: Errore durante creazione dataset da {dirpath}: {e}")
        return None, None


# Messaggio di conferma che le definizioni siano state caricate in memoria.
print("Funzioni per il Dataset definite.")

Funzioni per il Dataset definite.


### **Funzioni di Utilità**

funzioni per la gestione e l'automazione del flusso di lavoro. L'obiettivo è rendere lo script robusto, efficiente e facile da monitorare.

*   **`setup_logging`**: Invece di usare `print`, è stato implementato un sistema di **logging**. Questa scelta è strategica perché:
    1.  **Tracciabilità**: Ogni operazione viene registrata con un timestamp e un livello di gravità (INFO, ERROR) in un file di log persistente (`run.log`). Questo permette di analizzare l'esecuzione a posteriori e diagnosticare eventuali problemi, anche per processi che durano ore.
    2.  **Doppio Output**: I messaggi vengono mostrati sia in tempo reale sulla console, sia salvati su file, combinando immediatezza e persistenza.

*   **`is_already_processed` e `mark_processed`**: Queste due funzioni lavorano insieme per implementare un meccanismo di **caching dello stato di avanzamento**.

Durante l'inferenza, lo script potrebbe essere interrotto o rieseguito. Per evitare di ri-analizzare da capo file che hanno già richiesto tempo di calcolo, ogni file processato con successo viene "marcato" in un file di registro (`processed.log`).

Al successivo avvio, lo script controlla questo registro e salta i file già analizzati.

*   **`extract_frames`**: L'analisi di un video da parte di un modello di classificazione di immagini richiede la sua conversione in fotogrammi. Questa funzione automatizza il processo:

**Input**:
    
    1.  `video_path`: Il percorso del file video da processare.

    2.  `frame_output_base_dir`: La cartella principale dove verranno salvati i frame.

    3.  `interval_sec`: L'intervallo in secondi tra un frame estratto e il successivo (es. `1` per un frame al secondo).
*   **Output**:
    *   Una **lista** contenente i percorsi completi di tutti i file di immagine (i frame) che sono stati salvati su disco.

   *   `fps = cap.get(cv2.CAP_PROP_FPS) or 30.0`: La funzione legge i metadati del video per ottenere il suo framerate (FPS - Fotogrammi al Secondo).
    *   `step = max(1, int(fps * interval_sec))`. Calcola ogni quanti frame bisogna salvarne uno.
        *   **Es.**: Se un video ha 30 FPS e `interval_sec` è impostato a `2`, lo `step` sarà `60`. Questo significa che la funzione salverà un frame ogni 60 che ne legge, ottenendo di fatto un campionamento di un frame ogni due secondi.

4.  **Iterazione e Salvataggio**:
    *   Il ciclo `while True` scorre l'intero video, leggendo un frame alla volta con `cap.read()`.
    *   `if frame_count % step == 0`: L'operatore % fa da trigger. Solo quando il contatore dei frame è un multiplo esatto dello `step`, il frame corrente viene salvato.
    *   `out_file_path = ..._frame{saved_frame_index:05d}.jpg`: Il nome del file di output è formattato in teoria.
    *   `cv2.imwrite(...)`: Il frame viene salvato come file JPEG, con un livello di qualità esplicito del 90% per un buon compromesso tra qualità e dimensione del file.

5.  **Feedback e Pulizia delle Risorse**:
    *   `with tqdm(...) as pbar`: L'intera iterazione è avvolta da `tqdm`, che fornisce la **barra di avanzamento** in tempo reale.
    *   `cap.release()`: Al termine, si liberano le risorse


    (in realtà ho poi fatto il programma estrazionefile a parte per una questione di comodità)

In [ ]:
# Questa funzione imposta il sistema di logging per il programma.
# ( ** Il logging è superiore a `print` perché permette di avere messaggi formattati,
# con livelli di gravità diversi (INFO, WARNING, ERROR) e di scrivere
# contemporaneamente sia sulla console che su un file di log. )
# `output_dir: Path` è un'annotazione di tipo (type hint), indica che la funzione si aspetta un oggetto Path.
def setup_logging(output_dir: Path):
    # Definisce il percorso del file di log all'interno della cartella di output.
    log_file = output_dir / "run.log"

    # Questo ciclo è FONDAMENTALE in ambienti come Colab/Jupyter.
    # Se la cella viene eseguita più volte, `logging.basicConfig` non ha effetto dopo la prima volta.
    # Questo codice "resetta" il sistema di logging rimuovendo tutti i gestori (handler) esistenti,
    # garantendo che la configurazione venga riapplicata correttamente ogni volta.
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

    # Configura il logging a livello base.
    logging.basicConfig(
        level=logging.INFO,  # Imposta il livello minimo di messaggi da registrare (INFO e superiori).
        format="%(asctime)s %(levelname)s: %(message)s",  # Definisce il formato dei messaggi di log.
        datefmt='%Y-%m-%d %H:%M:%S',  # Definisce il formato del timestamp.
        # Definisce dove inviare i log:
        handlers=[
            logging.FileHandler(log_file, encoding='utf-8'),  # 1: a un file di testo (run.log).
            logging.StreamHandler()  # 2: alla console/output della cella.
        ]
    )
    # Registra un primo messaggio per confermare che il logging è attivo e indicare dove si trova il file.
    logging.info(f"Logging configurato. Log salvati in: {log_file}")


# Controlla se una chiave (che rappresenta un file) è già stata processata in precedenza.
# Questo evita di ri-elaborare file inutilmente, risparmiando tempo.
# `key: str` indica che si aspetta una stringa. `-> bool` indica che restituisce un booleano (True/False).
def is_already_processed(key: str) -> bool:
    try:
        # Se il file di log dei file processati non esiste, nessun file è stato processato.
        if not PROCESSED_LOG.exists(): return False
        # Apre il file in modalità lettura ('r').
        with open(PROCESSED_LOG, 'r', encoding='utf-8') as f:
           # Usa un'espressione generatore con `any()` per efficienza.
           # `any()` si ferma non appena trova una corrispondenza, senza leggere tutto il file se non necessario.
           # `line.strip()` rimuove spazi bianchi e caratteri di a capo dalla riga letta.
           return any(key == line.strip() for line in f)
    except Exception as e:
        # Se si verifica un errore durante la lettura del file, lo registra e assume che il file non sia processato.
        logging.error(f"Errore leggendo {PROCESSED_LOG}: {e}")
        return False


# Scrive una chiave nel file di log per marcarla come "processata".
def mark_processed(key: str):
    try:
        # Apre il file in modalità 'a' (append), che aggiunge testo alla fine del file senza cancellarlo.
        with open(PROCESSED_LOG, "a", encoding='utf-8') as f:
            # Scrive la chiave seguita da un carattere di a capo.
            f.write(key + "\n")
    except Exception as e:
        # Registra eventuali errori di scrittura.
        logging.error(f"Errore scrivendo su {PROCESSED_LOG}: {e}")


'''
# Estrae frame da un file video a un intervallo di tempo specificato.
# `-> list` indica che la funzione restituisce una lista (dei percorsi dei frame salvati).
def extract_frames(video_path: Path, frame_output_base_dir: Path, interval_sec: int) -> list:
    # Lista per memorizzare i percorsi dei frame salvati.
    saved_frames_paths = []
    # Crea una sottocartella specifica per i frame di questo video, per tenere tutto ordinato.
    # `video_path.stem` è il nome del file senza estensione.
    video_frame_dir = frame_output_base_dir / video_path.stem
    video_frame_dir.mkdir(parents=True, exist_ok=True)
    try:
        # Apre il file video usando OpenCV.
        cap = cv2.VideoCapture(str(video_path))
        # Controlla se il video è stato aperto correttamente.
        if not cap.isOpened():
            logging.error(f"Impossibile aprire il video: {video_path}")
            return []  # Restituisce una lista vuota in caso di fallimento.

        # Ottiene i fotogrammi al secondo (FPS) del video. Se fallisce, usa 30.0 come default.
        fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
        # Calcola ogni quanti frame leggerne uno, basandosi sull'intervallo in secondi.
        step = max(1, int(fps * interval_sec))
        # Inizializza i contatori.
        frame_count, saved_frame_index, total_frames = 0, 0, int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Usa `tqdm` per creare una barra di avanzamento che mostra il progresso dell'estrazione.
        with tqdm(total=total_frames, desc=f"Extracting frames from {video_path.name}") as pbar:
            while True:
                # Legge il frame successivo del video. `ret` è True se la lettura ha successo.
                ret, frame = cap.read()
                if not ret: break  # Esce dal ciclo se il video è terminato.

                # Controlla se il contatore di frame è un multiplo dello 'step' calcolato.
                if frame_count % step == 0:
                    # Costruisce il percorso del file di output per il frame.
                    # Il formato `:05d` assicura che il numero del frame abbia sempre 5 cifre (es. 00001, 00012).
                    out_file_path = video_frame_dir / f"{video_path.stem}_frame{saved_frame_index:05d}.jpg"
                    # Salva il frame come immagine JPEG con qualità 90.
                    if cv2.imwrite(str(out_file_path), frame, [cv2.IMWRITE_JPEG_QUALITY, 90]):
                        saved_frames_paths.append(out_file_path)
                        saved_frame_index += 1

                frame_count += 1
                pbar.update(1)  # Aggiorna la barra di avanzamento.

        # Rilascia l'oggetto video per liberare le risorse.
        cap.release()
        logging.info(f"Estratti {len(saved_frames_paths)} frame da {video_path.name}")
    except Exception as e:
        logging.error(f"Errore durante estrazione frame da {video_path}: {e}")
    # Restituisce la lista dei percorsi dei frame salvati.
    return saved_frames_paths
'''
    print("Funzioni di Utilità iniziali definite.")

### **`update_image_exif`**

Questa funzione è stata progettata per integrare il risultato della classificazione AI direttamente nell'immagine.


**1. Lettura Sicura dei Dati EXIF**
```python
img = Image.open(img_path)
exif_data = img.info.get('exif', b"")
exif_dict = piexif.load(exif_data) if exif_data else {'0th': {}}
```
**Non tutte le immagini JPEG contengono metadati EXIF.** Un approccio come `img.info['exif']` causerebbe un `KeyError` se i metadati fossero assenti.

*   **Implementazione**:
    *   `img.info.get('exif', b"")`: L'uso del metodo `.get()` per accedere ai dizionari in modo sicuro. Se la chiave `'exif'` non esiste, restituisce un valore predefinito (un oggetto `bytes` vuoto, `b""`) invece di generare un'eccezione. --> modificabile in futuro
    *   `if exif_data else ...`: L'operatore ternario controlla se `exif_data` non è vuoto. Solo in quel caso viene chiamato `piexif.load()`. Se è vuoto, viene creata una struttura `exif_dict` minima (`{'0th': {}}`) **per garantire che il codice successivo che tenta di scrivere in `exif_dict['0th']` non fallisca.** Questo rende la funzione **universale**, in grado di aggiungere metadati sia a immagini che ne sono già provviste, sia a immagini che ne sono completamente prive.

---

**2. Scrittura Strutturata del Tag**
```python
if '0th' not in exif_dict: exif_dict['0th'] = {}
exif_dict['0th'][piexif.ImageIFD.ImageDescription] = label.encode('utf-8')
```
Lo standard EXIF *è strutturato in sezioni chiamate IFD (Image File Directory).* La sezione principale è la `'0th'`. IMPORTANTE che ci sia
*   **Implementazione**:
    *   `if '0th' not in exif_dict...`: Questo controllo di sicurezza gestisce il caso in cui un file abbia dati EXIF ma manchi della sezione principale, prevenendo un `KeyError`.

    *   `piexif.ImageIFD.ImageDescription`: Invece di usare il codice del tag è 270, penso che sia più chiaro usare una costante nominata(?).
    *   `.encode('utf-8')`: Lo standard EXIF richiede che i valori testuali siano memorizzati come `bytes`, non come stringhe Python. L'uso di `.encode('utf-8')` serve garantire la compatibilità, poiché UTF-8 è in grado di rappresentare quasi tutti i caratteri e i simboli internazionali.

---

**3. Ottimizzazione e Compatibilità**
```python
exif_dict['thumbnail'] = None
exif_bytes = piexif.dump(exif_dict)
img.save(img_path, exif=exif_bytes, quality=95)
```
Ovviamente i dati vanno puliti in modo sicuro
*   **Implementazione**:
    *   `exif_dict['thumbnail'] = None`: Le thumbnail incorporate nei dati EXIF sono una fonte comune di problemi. Alcuni software le gestiscono male, e possono portare a corruzione dei metadati. Rimuoverle (`None`) è una misura preventiva che aumenta l'affidabilità del file generato e ne riduce leggermente le dimensioni.
    *   `piexif.dump()`: Questa funzione esegue l'operazione inversa, riconvertendo il dizionario Python, ora modificato, nel formato binario richiesto dallo standard EXIF.
    *   `img.save(..., quality=95)`: Quando si salva un'immagine JPEG, è necessario specificare un livello di qualità. Omettere questo parametro farebbe sì che Pillow utilizzi un valore predefinito che potrebbe degradare la qualità visiva dell'immagine. Impostare esplicitamente `quality=95` è una scelta per preservare la qualità dell'immagine originale il più possibile.

  *  un file JPEG non può contenere un dizionario Python. Richiede un formato binario molto specifico, una sequenza di bytes strutturata secondo lo standard EXIF.

---

**4. Gestione delle Risorse**
```python
img.close()
```
Ogni volta che un programma apre un file, occupa una piccola porzione di risorse di sistema (file handle).
*   **Implementazione**: `img.close()` chiude esplicitamente il file, assicurando che le risorse vengano liberate. (meglio per programmi che devono lavorare con tanti file rispetto a garbage collection)

In [ ]:
# Aggiorna i metadati EXIF di un'immagine per includere la label predetta.
def update_image_exif(img_path: Path, label: str):
    try:
        # Apre l'immagine con Pillow
        img = Image.open(img_path)
        # Tenta di leggere i dati EXIF esistenti. Se non ce ne sono, usa un byte string vuoto
        exif_data = img.info.get('exif', b"")
        # Carica i dati EXIF in un dizionario. Se non ci sono dati, crea un dizionario vuoto
        exif_dict = piexif.load(exif_data) if exif_data else {'0th': {}}
        # Si assicura che il sotto-dizionario '0th' (che contiene i tag principali) esista
        if '0th' not in exif_dict: exif_dict['0th'] = {}
        # Imposta il campo 'ImageDescription' con la nostra etichetta. `piexif` richiede che la stringa sia codificata.
        exif_dict['0th'][piexif.ImageIFD.ImageDescription] = label.encode('utf-8')
        # Rimuove la thumbnail per evitare problemi di compatibilità e ridurre la dimensione del file.
        exif_dict['thumbnail'] = None
        # Converte di nuovo il dizionario in formato bytes, pronto per essere scritto.
        exif_bytes = piexif.dump(exif_dict)
        # Salva l'immagine sovrascrivendo quella vecchia, ma con i nuovi dati EXIF.
        img.save(img_path, exif=exif_bytes, quality=95)
        # Chiude il file immagine.
        img.close()
    except Exception as e:
        # Se qualcosa va storto (es. file corrotto, formato non supportato), registra un avviso senza bloccare lo script.
        logging.warning(f"UPDATE_EXIF: Errore per {img_path}: {e}")


### **`calculate_sharpness`**

Quando si estraggono frame da un video, specialmente se girato a mano, è inevitabile ottenere immagini di qualità variabile. Alcune saranno nitide, altre saranno mosse o sfocate a causa del movimento della telecamera. Per garantire che il modello di classificazione riceva solo input di alta qualità (principio Garbage In, Garbage Out), è stata implementata la funzione `calculate_sharpness`.

In pratica assegna **un punteggio numerico oggettivo alla nitidezza di un'immagine**. Questo punteggio permette di filtrare automaticamente i frame di bassa qualità, scartando quelli che cadono al di sotto di una soglia predefinita (`SHARPNESS_THRESHOLD`).

Assicura che i frame selezionati come rappresentativi di una scena siano anche di una qualità buona, migliorando l'affidabilità complessiva del sistema.

* **le immagini nitide contengono molti bordi e dettagli ad alta frequenza, mentre le immagini sfocate ne sono prive.** Il metodo quantifica questa idea in tre passaggi:

1.  **Conversione in Scala di Grigi (`cv2.cvtColor`)**:
    * Trasforma l'immagine a colori in un'immagine in bianco e nero.
    * La nitidezza è una proprietà legata ai cambiamenti di luminosità (edges), non ai colori. Lavorare su un singolo canale di colore invece di tre rende il calcolo molto più semplice ed efficiente, senza perdita di informazioni pertinenti per questo specifico compito.

2.  **Applicazione dell'Operatore Laplaciano (`cv2.Laplacian`)**:
    * Applica un filtro Laplaciano all'immagine in scala di grigi. Questo filtro è un operatore matematico che calcola la derivata seconda dell'immagine. In teoria dovrebbe **rilevare e accentuare le zone in cui i pixel cambiano di intensità molto rapidamente** (bordi e ai dettagli fini).
    *  Se potessimo vedere l'immagine dopo il filtro Laplaciano:
        *   In un'**immagine nitida**, i bordi degli oggetti apparirebbero come linee bianche e nere molto intense su uno sfondo grigio.
        *   In un'**immagine sfocata**, dove i bordi sono sfumati, l'output sarebbe un'immagine per lo più grigia e uniforme, con poche o nessuna linea netta.

3.  **Calcolo della Varianza (`.var()`)**:
    * Calcola la varianza dei valori dei pixel dell'immagine filtrata dal Laplaciano. La varianza è una misura statistica di quanto i valori in un insieme di dati si discostano dalla media.

        *   Nell'**immagine nitida** filtrata, ci sono molti pixel con valori estremi (bianco e nero), molto lontani dal grigio medio. Questo porta a una **varianza alta**.
        *   Nell'**immagine sfocata** filtrata, la maggior parte dei pixel è vicina al grigio medio. Questo porta a una **varianza bassa**.
        
*   `if image_cv2 is None or image_cv2.size == 0`: Previene un errore nel caso in cui venga passata un'immagine non valida o vuota, restituendo un punteggio di `0.0`.


In [ ]:
# Calcola un punteggio di nitidezza per un'immagine.
# Utile per filtrare i frame mossi o sfocati.
def calculate_sharpness(image_cv2):
    try:
        # Controllo di sicurezza per evitare errori se l'immagine non è valida.
        if image_cv2 is None or image_cv2.size == 0: return 0.0
        # Converte l'immagine in scala di grigi, perché la nitidezza è legata ai contorni, non ai colori.
        gray = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2GRAY)
        # Applica l'operatore Laplaciano, che evidenzia le regioni con rapidi cambi di intensità (i contorni).
        # Calcola la varianza del risultato. Un'immagine nitida avrà molti contorni forti e quindi un'alta varianza.
        # Un'immagine sfocata avrà bassa varianza.
        return cv2.Laplacian(gray, cv2.CV_64F).var()
    except Exception as e:
        # Gestisce eventuali errori durante le operazioni di OpenCV.
        logging.error(f"CALCULATE_SHARPNESS: Errore: {e}")
        return 0.0

print("Funzioni di nitidezza definita.")

Funzioni di nitidezza definita.


### **`build_model`**

#### **1. `EfficientNetV2B0`**

```python
backbone = EfficientNetV2B0(
    weights="imagenet",
    include_top=False,
    # ...
)
```
*   Dopo diversi tentativi con altre reti neurali preaddestrate --> `EfficientNetV2B0`(ImageNet)
*   **`include_top=False`**: Si rimuove la parte finale del modello originale, che è specializzata nel classificare le 1000 classi di ImageNet. In questo modo, conserviamo solo la parte utile del modello: **l'estrattore di feature**.

#### **2. Congelamento del Backbone: `backbone.trainable = False`**

* Nella prima fase di addestramento, si sfrutta in teoria la conoscenza della backbone senza andare ad alterala. Se si addestra tutto il modello da subito con i nostri dati (relativamente pochi), rischieremmo di rovinare i pesi calibrati di ImageNet.
* Congelando il backbone, diciamo a TensorFlow di non modificare i suoi pesi. Questo ci permette di addestrare solo la nuova testa che viene aggiunta dopo.

#### **3.**

*   **`layers.GlobalAveragePooling2D`**: Questo layer prende la mappa di feature prodotta dal backbone **e la condensa in un singolo vettore**. Serve per ridurre il numero di parametri. (in teoria riduce anche l'overfitting)
*   **`layers.Dropout(0.3)`**: Durante l'addestramento, "spegne" casualmente il 30% dei neuroni. Questo costringe la rete a non fare eccessivo affidamento su specifici percorsi neurali, ma a imparare rappresentazioni più distribuite e generalizzabili.
*   **`layers.Dense(num_classes, activation="softmax")`**: Questo è il classificatore finale. È un layer con un neurone per ogni classe che vogliamo predire. L'attivazione **`softmax`** trasforma i punteggi grezzi del modello in un vettore di probabilità (es. `[0.95, 0.05]`), la cui somma è 1, rendendo l'output facilmente interpretabile.

#### **4. Compilazione del Modello**

* **`optimizer=optimizers.Adam(...)`** : Adam è una scelta standard per la maggior parte dei problemi.

* **`loss="categorical_crossentropy"`** : È la funzione di loss ideale per problemi di classificazione multi-classe.

In [ ]:
# Funzione per costruire il modello EfficientNetV2
# Questa funzione definisce l'intera architettura della rete neurale che verrà addestrata.
# `num_classes: int` indica che si aspetta un intero (il numero di categorie da predire).
# `-> models.Model` indica che restituisce un oggetto Modello di Keras.
def build_model(num_classes: int) -> models.Model:
    """Costruisce un modello di classificazione basato su EfficientNetV2B0 pre-addestrato."""

    # Carica il modello EfficientNetV2B0(CNN)
    # Questo modello funge da "backbone" (spina dorsale) per l'estrazione delle feature.
    backbone = EfficientNetV2B0(
        weights="imagenet",
        include_top=False,
        input_shape=(*IMG_SIZE, 3)
    )

    # Congela tutti i pesi del backbone
    # Impostando `trainable = False`, diciamo a Keras di non aggiornare i pesi del backbone
    # durante la prima fase di addestramento. In questo modo, addestriamo solo la "testa" che aggiungiamo noi.
    backbone.trainable = False

    # Costruzione della "Testa" di Classificazione
    # Questi sono i nuovi layer che mettiamo in cima al backbone.

    # 1. Prende l'output del backbone (una mappa di feature multidimensionale).
    # 2. `GlobalAveragePooling2D` calcola la media di ogni mappa di feature, trasformandola in un singolo numero.
    #    Questo riduce drasticamente il numero di parametri e rende il modello più robusto a piccole traslazioni dell'oggetto.
    #    L'output è un vettore di feature per ogni immagine.
    x = layers.GlobalAveragePooling2D(name="gap")(backbone.output)

    # 3. `Dropout` è un'altra tecnica di regolarizzazione fondamentale. Durante il training, "spegne" casualmente
    #    il 30% dei neuroni. Questo costringe la rete a non fare troppo affidamento su singoli neuroni
    #    e a imparare rappresentazioni più distribuite e robuste.
    x = layers.Dropout(0.3, name="dropout")(x)

    # 4. `Dense` è un layer "fully-connected" standard. Questo è il nostro classificatore finale.
    #    - `num_classes`: Il numero di neuroni in output, uno per ogni classe che vogliamo predire.
    #    - `activation="softmax"`: La funzione di attivazione Softmax trasforma i punteggi grezzi (logits) del layer
    #      in un vettore di probabilità, dove la somma di tutti gli elementi è 1.
    outputs = layers.Dense(num_classes, activation="softmax", name="predictions")(x)

    # Crea l'oggetto Modello finale, specificando gli input (quelli del backbone) e gli output (quelli del nostro classificatore).
    model = models.Model(backbone.input, outputs, name="efficientnetv2b0_finetuned")

    # Compilazione del Modello (configura il modello per il training)
    model.compile(
        # `optimizer`: L'algoritmo che aggiorna i pesi del modello. Adam è una scelta molto comune e robusta.
        # Usiamo il learning rate definito per la fase di training della testa.
        optimizer=optimizers.Adam(learning_rate=LEARN_HEAD_LR),

        # `loss`: La funzione di perdita. Misura quanto le previsioni del modello sono sbagliate.
        # `categorical_crossentropy` è la scelta standard per problemi di classificazione multi-classe
        # quando le etichette sono in formato one-hot (come fa `label_mode="categorical"`).
        loss="categorical_crossentropy",

        # `metrics`: Metriche da monitorare durante il training. L'accuracy (precisione) è la più comune.
        metrics=["accuracy"]
    )

    logging.info(f"Modello EfficientNetV2B0 costruito e compilato per il training della testa.")
    return model

### **`train`**
Il codice inizia con una gestione dei dataset di training e validazione.
* Il modello supporta due scenari:
    1.  **Split Separato**: Se viene fornita una cartella di validazione (`val_dir`), questa viene usata interamente per la validazione. Questo è l'approccio migliore perché garantisce che i dati di validazione siano completamente disgiunti da quelli di training fin dall'inizio.
    2.  **Split Automatico**: Se la cartella di validazione non viene fornita, lo script riserva automaticamente una porzione (il 20%) dei dati di training per la validazione. Questa è un'opzione comoda per esperimenti rapidi.
* La data augmentation (`augment=True`) viene applicata **solo** al set di training. I dati di validazione non devono mai essere aumentati, poiché devono rappresentare i dati reali su cui vogliamo misurare le performance del modello in modo oggettivo ad ogni epoca.


*  Si utilizza un learning rate relativamente alto (`LEARN_HEAD_LR`) per accelerare questa fase iniziale di apprendimento.

**Fine tuning**
* Scongelare il modello intero (`backbone.trainable = True`) e continuare l'addestramento per permettere a tutti i layer di adattarsi al dataset.
* Questo permette al backbone di specializzarsi ulteriormente, passando da un estrattore di feature generico a uno ottimizzato per distinguere tra le varie classi.
* È **fondamentale** ricompilare il modello e usare un **learning rate molto più basso** (`LEARN_FULL_LR`). Questo previene modifiche drastiche ai pesi pre-addestrati, evitando *il catastrophic forgetting*.

#### **3. Automazione e Sicurezza con i Callbacks**

Per rendere il processo meno dipendente dalla supervisione manuale, sono stati utilizzati dei `Callbacks` di Keras.

*   **`ModelCheckpoint`**:
    * Salvano automaticamente il modello alla fine di ogni epoca, ma **solo se** le sue performance sul set di validazione (`monitor='val_accuracy'`) sono migliorate.
    *   **Implementazione**: Sono stati creati due checkpoint separati, uno per ogni fase, per salvare il miglior modello della fase di head training(lr alto) e il miglior modello della fase di fine tuning. Questo garantisce che, anche se le ultime epoche dovessero peggiorare, conserveremo sempre la versione più performante del modello.

*   **`EarlyStopping`**:
    * Monitorare la perdita sul set di validazione (`monitor='val_loss'`) e interrompere automaticamente l'addestramento se questa non migliora per un numero definito di epoche (`patience=5`).
    *   **Vantaggi**: Previene l'overfitting e risparmia tempo di calcolo, fermando il processo quando non si stanno più ottenendo benefici.
    *   **`restore_best_weights=True`**: Garantisce che, quando il training si ferma, i pesi del modello vengano ripristinati allo stato della migliore epoca, non a quelli dell'ultima.

1.  **`place_model.keras`**: Il modello finale, addestrato e pronto per essere utilizzato per l'inferenza.
2.  **`class_indices.csv`**: Un file di mappatura che associa i nomi delle classi (es. "economia_interno") al loro indice numerico (es. 0). Questo file permette di tradurre l'output numerico del modello (che predice un indice) in un'etichetta leggibile dall'uomo.

In [ ]:
# Questa funzione incapsula l'intero processo di addestramento del modello.
# Prende un oggetto `args` (che simula gli argomenti da riga di comando)
# contenente tutte le configurazioni necessarie.
def train(args):
    """Esegue il ciclo di training completo."""
    # Inizializza il sistema di logging per questa esecuzione.
    setup_logging(args.output)
    logging.info("Inizio Processo di Training")

    # Preparazione dei Dataset
    # Converte i percorsi da stringa a oggetti Path per una gestione più semplice.
    train_dir_path = Path(args.train_dir)
    # Gestisce il caso in cui un percorso di validazione separato non sia fornito.
    val_dir_path = Path(args.val_dir) if args.val_dir else None

    # Controlla se è stata fornita una directory di validazione separata e se esiste.
    if val_dir_path and val_dir_path.is_dir():
        # Se sì, crea il dataset di training e quello di validazione da due directory distinte.
        logging.info(f"Uso directory di validazione separata: {val_dir_path}")
        # Per il training: mescola i dati (shuffle=True) e applica data augmentation (augment=True).
        train_ds, class_names = make_dataset(train_dir_path, shuffle=True, augment=True)
        # Per la validazione: non mescola (per avere risultati consistenti) e non applicare augmentation.
        val_ds, _ = make_dataset(val_dir_path, shuffle=False) # (** L'underscore `_` ignora i nomi delle classi, che già abbiamo )
    else:
        # Se non c'è una cartella di validazione, crea entrambi i set partendo dalla sola cartella di training,
        # riservando una frazione dei dati (es. 20%) per la validazione.
        logging.info(f"Uso 20% split da {train_dir_path} per validazione.")
        # `subset='training'` e `validation_split=0.2` dicono a `make_dataset` di creare il set di training con l'80% dei dati.
        train_ds, class_names = make_dataset(train_dir_path, shuffle=True, subset='training', validation_split=0.2, augment=True)
        # `subset='validation'` crea il set di validazione con il restante 20%.
        val_ds, _ = make_dataset(train_dir_path, shuffle=False, subset='validation', validation_split=0.2)

    # Controllo di sicurezza: se la creazione di uno dei dataset è fallita, interrompe il training.
    if not all([train_ds, val_ds, class_names]):
        logging.error("Creazione dataset fallita. Training interrotto.")
        return # Esce dalla funzione.

    # Recupera il numero di classi e le stampa per verifica.
    num_classes = len(class_names)
    logging.info(f"Trovate {num_classes} classi: {class_names}")

    # Costruisce l'architettura del modello chiamando la funzione definita in precedenza.
    model = build_model(num_classes)

    # Definizione dei Callbacks
    # I callbacks sono oggetti che eseguono azioni specifiche in vari momenti del training (es. alla fine di ogni epoca).

    # `ModelCheckpoint`: Salva il modello.
    checkpoint_head = tf.keras.callbacks.ModelCheckpoint(
        filepath=str(args.output / "model_head_best.keras"), # Dove salvare il file.
        monitor='val_accuracy',          # Metrica da monitorare per decidere se il modello è "migliore".
        save_best_only=True,             # Salva solo se la metrica monitorata è migliorata.
        mode='max',                      # La 'val_accuracy' deve essere massimizzata.
        verbose=1                        # Stampa un messaggio quando il modello viene salvato.
    )
    # Un secondo checkpoint per la fase di fine-tuning.
    checkpoint_fine = tf.keras.callbacks.ModelCheckpoint(
        filepath=str(args.output / "model_fine_tuned_best.keras"),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )

    # `EarlyStopping`: Interrompe il training se il modello smette di migliorare.
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',              # Monitora la perdita sul set di validazione.
        patience=5,                      # Numero di epoche da attendere senza miglioramenti prima di fermarsi.
        restore_best_weights=True,       # Al termine, ripristina i pesi del modello alla migliore epoca.
        verbose=1                        # Stampa un messaggio quando il training viene interrotto.
    )

    # FASE 1: Training della sola testa del modello
    logging.info(f"FASE 1: Training testa per {args.epochs_head} epoche")
    # Avvia il training vero e proprio.
    model.fit(
        train_ds,                        # Dati di training.
        validation_data=val_ds,          # Dati di validazione per monitorare le performance.
        epochs=args.epochs_head,         # Numero di epoche per questa fase.
        callbacks=[checkpoint_head]      # Applica il callback per salvare il miglior modello di questa fase.
    )

    # Dopo la prima fase, carica esplicitamente il miglior modello salvato dal ModelCheckpoint.
    # Questo assicura che si stia partendo dal miglior stato possibile per il fine-tuning,
    # anche se l'ultima epoca non era la migliore.
    logging.info("Caricamento del miglior modello dal training della testa.")
    model = models.load_model(args.output / "model_head_best.keras")

    # FASE 2: Fine-tuning dell'intero modello
    # `model.layers[0]` si riferisce al primo layer del nostro modello, che è il backbone EfficientNetV2.
    model.layers[0].trainable = True # Scongela i pesi del backbone, rendendoli aggiornabili.

    # Ricompila il modello. Questo è NECESSARIO dopo aver cambiato lo stato `trainable` di un layer.
    # Usiamo un learning rate molto più basso per il fine-tuning.
    model.compile(optimizer=optimizers.Adam(learning_rate=args.learn_full_lr),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    logging.info(f"Modello ricompilato per fine-tuning (LR={args.learn_full_lr})")

    logging.info(f"FASE 2: Fine-tuning per max {args.epochs_fine} epoche")
    # Avvia la seconda fase di training.
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=args.epochs_fine,
        # Usa sia il checkpoint per salvare il miglior modello di questa fase,
        # sia l'early stopping per fermarsi se non ci sono più miglioramenti.
        callbacks=[checkpoint_fine, early_stopping]
    )

    # Salvataggio Finale
    # Al termine di tutto il processo, salva il modello finale (che, grazie a `restore_best_weights`,
    # dovrebbe essere il migliore della fase di fine-tuning) in un percorso standard.
    final_model_path = args.output / "place_model.keras"
    logging.info(f"Salvataggio modello finale in: {final_model_path}")
    model.save(final_model_path)

    # Salva la mappatura tra i nomi delle classi (es. "Cucina") e i loro indici numerici (es. 0).
    # Questo file è FONDAMENTALE per l'inferenza, per poter tradurre l'output del modello (un indice)
    # in un'etichetta leggibile.
    class_indices_path = args.output / "class_indices.csv"
    # Crea un DataFrame pandas e lo salva come CSV.
    pd.DataFrame({"class_name": class_names, "index": list(range(num_classes))}).to_csv(class_indices_path, index=False)
    logging.info(f"Mappatura classi salvata in: {class_indices_path}")
    logging.info("Processo di Training Concluso")

# Messaggio di conferma che la funzione `train` è stata definita.
print("Funzione di Training definita.")

Funzione di Training definita.


In [ ]:
# Blocco di Pulizia Opzionale
# Questo blocco di codice al momento non utilizzato,
# Serve per resettare lo stato del training, eliminando i modelli e i log delle esecuzioni precedenti.
# È utile quando si vuole essere sicuri di iniziare un addestramento completamente da zero.

# print("Pulizia dei vecchi file di modello e log in corso...")
# # Lista dei nomi dei file di modello che vengono creati durante il training.
# old_model_files = ["model_head_best.keras", "model_fine_tuned_best.keras", "place_model.keras"]
# # Itera sulla lista per cancellare ogni file.
# for f_name in old_model_files:
#     # Costruisce il percorso completo del file.
#     # `exists()` controlla se il file esiste prima di provare a cancellarlo, per evitare errori.
#     if (DEFAULT_OUTPUT / f_name).exists():
#         # `.unlink()` è il metodo dell'oggetto Path per cancellare un file.
#         (DEFAULT_OUTPUT / f_name).unlink()
# # Controlla anche l'esistenza del file di log dei file processati.
# if PROCESSED_LOG.exists():
#     # E lo cancella se esiste.
#     PROCESSED_LOG.unlink()
# print("Pulizia completata.")


# Configurazione per il Training
# Qui creiamo un oggetto che conterrà tutti i parametri da passare alla funzione `train`.
# Usiamo `SimpleNamespace` come un modo semplice e veloce per creare un oggetto "contenitore"
# che si comporta in modo simile all'output di `argparse`, permettendoci di accedere ai valori
# con la notazione punto (es. `args.train_dir`).

args_train_config = SimpleNamespace(
    # Passiamo i percorsi delle directory di training e validazione.
    # Vengono convertiti in stringa (`str()`) perché alcune librerie più vecchie
    # potrebbero non accettare direttamente gli oggetti Path. È una buona pratica di compatibilità.
    train_dir=str(DEFAULT_TRAIN_DIR),
    val_dir=str(DEFAULT_VAL_DIR),

    # Il percorso della directory di output.
    output=DEFAULT_OUTPUT,

    # I parametri numerici (iperparametri) definiti nella cella di configurazione globale.
    epochs_head=EPOCHS_HEAD,
    epochs_fine=EPOCHS_FINE,
    learn_head_lr=LEARN_HEAD_LR,
    learn_full_lr=LEARN_FULL_LR
)


# ESECUZIONE DEL TRAINING
try:
    # Chiama la funzione `train`, passandole l'oggetto di configurazione appena creato
    # Da questo momento, il controllo passa alla funzione `train` che eseguirà
    # il caricamento dei dati, la costruzione del modello e le due fasi di addestramento
    train(args_train_config)

# Cattura qualsiasi eccezione (`Exception`) che potrebbe verificarsi durante l'esecuzione
# della funzione `train`. Questo previene un crash improvviso del notebook e fornisce
# informazioni utili per il debug.
except Exception as e:
    # Stampa un messaggio di errore chiaro e conciso.
    print(f"ERRORE INASPETTATO DURANTE IL TRAINING: {e}")
    # Importa il modulo `traceback` per ottenere maggiori dettagli sull'errore.
    import traceback
    # `traceback.print_exc()` stampa la "traccia dello stack", che mostra esattamente
    # in quale punto del codice e attraverso quale catena di chiamate di funzioni
    # si è verificato l'errore. È uno strumento di debug potentissimo.
    traceback.print_exc()

2025-09-10 01:36:04 INFO: Logging configurato. Log salvati in: /content/drive/MyDrive/PROGETTO/output/run.log
2025-09-10 01:36:05 INFO: Inizio Processo di Training
2025-09-10 01:36:05 INFO: Uso directory di validazione separata: /content/drive/MyDrive/PROGETTO/dataset/val


Found 7634 files belonging to 3 classes.
Found 338 files belonging to 3 classes.


2025-09-10 01:36:23 INFO: Trovate 3 classi: ['economia_esterno', 'economia_interno', 'stum_interno']


24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


2025-09-10 01:36:26 INFO: Modello EfficientNetV2B0 costruito e compilato per il training della testa.
2025-09-10 01:36:26 INFO: FASE 1: Training testa per 10 epoche


Epoch 1/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.8937 - loss: 0.2790 
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to /content/drive/MyDrive/PROGETTO/output/model_head_best.keras
239/239 ━━━━━━━━━━━━━━━━━━━━ 3483s 14s/step - accuracy: 0.8941 - loss: 0.2783 - val_accuracy: 1.0000 - val_loss: 0.0198
Epoch 2/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.0076
Epoch 2: val_accuracy did not improve from 1.00000
239/239 ━━━━━━━━━━━━━━━━━━━━ 285s 1s/step - accuracy: 1.0000 - loss: 0.0076 - val_accuracy: 1.0000 - val_loss: 0.0079
Epoch 3/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.0033
Epoch 3: val_accuracy did not improve from 1.00000
239/239 ━━━━━━━━━━━━━━━━━━━━ 291s 1s/step - accuracy: 1.0000 - loss: 0.0033 - val_accuracy: 1.0000 - val_loss: 0.0051
Epoch 4/10
239/239 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.0021
Epoch 4: val_accuracy did not improve from 1.00000
239/239 ━━━━━━━━━━━

### **`infer`**

`infer` prende il modello addestrato e lo utilizza per classificare nuove immagini e video. È stata progettata per non fare solo la previsione, ma gestire anche l'organizzazione dei risultati e l'arricchimento dei dati di output.

#### **1. Setup**

Prima di iniziare, la funzione esegue controlli preliminari essenziali per la stabilità del processo:

*   **Caricamento degli Artefatti**: Carica i file prodotti dalla fase di training: il modello salvato (`place_model.keras`) e la mappatura delle classi (`class_indices.csv`).
*   **Controlli di Esistenza**: Verifica che questi file esistano. Invece di generare un errore, invita l'utente a eseguire prima il training.
*   **Ordinamento Classi**: l'ordine delle classi (`class_names`) deve corrispondere esattamente agli indici di output del modello (dove l'output `0` corrisponde alla prima classe, `1` alla seconda, etc.). Il codice assicura questo ordinamento leggendo il CSV e ordinandolo per la colonna `index`.

#### **2. Processo di Inferenza su Immagini Statiche**

qui si gestisce la classificazione delle singole immagini.

* Per ogni immagine, viene chiamata la funzione `load_and_preprocess_img_infer`. Questa funzione esegue gli stessi identici passaggi di pre-processing usati durante l'addestramento (ridimensionamento a `IMG_SIZE` e normalizzazione con `preprocess_input`). **Questa coerenza è molto importante**: il modello deve vedere i nuovi dati nello stesso identico formato dei dati su cui è stato addestrato.
*   **Predizione**:
    *   `tf.expand_dims`: Il modello si aspetta di ricevere un "batch" di immagini, non una singola immagine. **Questa funzione aggiunge una dimensione extra, trasformando un'immagine singola in un batch di dimensione 1.** --> Quando carichi e pre-processi una singola immagine, ottieni un tensore con questa forma a 3 dimensioni: (224, 224, 3). Se provi a dare questo tensore direttamente al modello con model.predict(...), TensorFlow ti da errore.
    *   `model.predict()`: Esegue la predizione vera e propria. L'output è un vettore di probabilità (es. `[0.05, 0.95]`).
    *   `np.argmax()`: Trova l'indice del valore più alto nel vettore di probabilità, che corrisponde all'indice della classe predetta.
    
## **Filtraggio per Confidenza**
Una previsione viene **accettata solo se la sua confidenza è superiore a una soglia (50% --> che comunque verrà alzata, per adesso funziona)**. Questo dovrebbe servire per ridurre i falsi positivi. Se il modello non è "sicuro" della sua previsione, il risultato viene scartato (ma comunque registrato nei log per analisi future).
*   **Organizzazione dell'Output**: Le immagini accettate vengono:
    1.  Copiate in una sottocartella di output il cui nome è la classe predetta (es. `/output/classified_images/economia_interno/`). (cosa che verrà modificata/eliminata una volta che si lavorerà con il NAS)
    2.  Arricchite con i metadati EXIF tramite `update_image_exif`:
    `update_image_exif(dest_path, label)`
    3.  Registrate con tutti i dettagli in un file CSV (`results.csv`) per una analisi successiva.

#### **3. Processo di Inferenza su Video**

*   **Da Video a Frame**: Il primo passo è scomporre il video in una sequenza di fotogrammi usando la funzione `extract_frames`. (per i video usati per ricavare le immagini che ho usato per addestrare il modello, ho fatto un programma a parte, estrazioneframe.ipynb, è su github)
*   **Predizione in Batch**: Per massimizzare l'efficienza, tutti i frame estratti vengono pre-processati e poi dati in pasto al modello in **un unico batch**. Questo sfrutta la capacità della GPU di eseguire calcoli in parallelo ed è molto più veloce che processare i frame uno per uno.
*  **Conteggio delle Occorrenze**: Viene calcolato quante volte ogni classe (es. `economia_interno`, `stum_interno`) è stata predetta.
    * **Filtro a Soglia**: Viene definita una soglia di presenza (es. 10%). Ogni classe il cui conteggio supera questa soglia rispetto al numero totale di frame viene aggiunta a una lista di "luoghi riconosciuti".
    *  **Output Multi-Etichetta**: Il risultato finale per il video non è una singola etichetta, ma un elenco di tutte le classi che hanno superato la soglia. Ad esempio, `economia_interno, stum_interno`.

Quindi, Se un video mostra due aule diverse, il sistema le riporterà entrambe, fornendo un riassunto completo del contenuto.
    *   **Implementazione**: `np.bincount(pred_indices).argmax()` è un modo in NumPy per eseguire questo conteggio e trovare la classe vincente.

Sui frame candidati, viene calcolato il punteggio di nitidezza (`calculate_sharpness`).
    *   **Output Finale**: Vengono salvati solo i **top N frame** (definito da `FRAMES_PER_PLACE_PER_VIDEO`), ordinati per nitidezza decrescente. Questo garantisce che l'output non sia solo una rappresentazione corretta del contenuto del video.

*   **Tracciamento e Pulizia**: Anche per i video, i risultati aggregati vengono salvati nel file CSV. Infine, la cartella temporanea contenente tutti i frame estratti viene eliminata per liberare spazio.

In [ ]:
# La funzione `infer` è responsabile di usare il modello addestrato per fare previsioni
# su nuovi dati (immagini e video).
def infer(args):
    """Esegue inferenza su immagini e video."""
    # Imposta il logging per questa esecuzione.
    setup_logging(args.output)
    logging.info("Inizio Processo di Inferenza")

    # Caricamento del Modello e delle Classi
    model_path = args.output / "place_model.keras"
    class_indices_path = args.output / "class_indices.csv"

    if not model_path.exists() or not class_indices_path.exists():
        logging.error(f"Modello ({model_path}) o indici classi ({class_indices_path}) non trovati.")
        logging.error("Esegui prima il training per creare il modello.")
        return

    try:
        model = models.load_model(model_path)
        class_names_df = pd.read_csv(class_indices_path)
        class_names = class_names_df.sort_values("index")["class_name"].tolist()
        logging.info(f"Modello e {len(class_names)} classi caricate: {class_names}")
    except Exception as e:
        logging.error(f"Errore durante il caricamento del modello o delle classi: {e}")
        return

    CLASSIFIED_IMG_OUTPUT_DIR = args.output / "classified_images"
    CLASSIFIED_IMG_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    logging.info(f"Le immagini classificate verranno salvate in: {CLASSIFIED_IMG_OUTPUT_DIR}")

    with open(args.csv_path, "a", newline="", encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=["SourceType", "PredictedPlace", "Confidence", "OriginalPath", "OutputPath", "IsVideo", "Sharpness"])
        if csv_file.tell() == 0:
            writer.writeheader()

        def load_and_preprocess_img_infer(path_str):
            try:
                img_bytes = tf.io.read_file(path_str)
                img = tf.image.decode_image(img_bytes, channels=3, expand_animations=False)
                img = tf.image.resize(img, IMG_SIZE, method='bilinear')
                img_preprocessed = preprocess_input(img)
                return img_preprocessed
            except Exception as e:
                logging.warning(f"Errore caricamento immagine {path_str}: {e}")
                return None

        # BLOCCO DI INFERENZA SULLE IMMAGINI STATICHE
        logging.info(f"Inizio Inferenza su Immagini Statiche in: {args.img_dir}")
        img_dir = Path(args.img_dir)
        if img_dir.is_dir():
            image_files = sorted( list(img_dir.glob('*.jpg'))   + list(img_dir.glob('*.jpeg'))  + list(img_dir.glob('*.png'))   + list(img_dir.glob('*.JPG'))   + list(img_dir.glob('*.JPEG'))  + list(img_dir.glob('*.PNG')))
            logging.info(f"Trovate {len(image_files)} immagini da processare.")

            for img_path in tqdm(image_files, desc="Processing Images"):
                key = f"IMG::{img_path.resolve()}"
                if is_already_processed(key):
                    logging.info(f"Immagine già processata, saltata: {img_path.name}")
                    continue

                img_tensor = load_and_preprocess_img_infer(str(img_path))
                if img_tensor is None:
                    continue


            # Il modello si aspetta un batch di immagini, non una singola immagine.
            # `tf.expand_dims` aggiunge una dimensione all'inizio, trasformando la forma
            # da (224, 224, 3) a (1, 224, 224, 3).
                img_batch = tf.expand_dims(img_tensor, axis=0)
                prediction = model.predict(img_batch, verbose=0)[0]
                pred_index = np.argmax(prediction)
                confidence = float(np.max(prediction))

                if confidence >= 0.50:
                    label = class_names[pred_index]
                    logging.info(f"Immagine: {img_path.name} -> Classe: {label} (Conf: {confidence:.3f}) -> ACCETTATA")
                    dest_dir = CLASSIFIED_IMG_OUTPUT_DIR / label
                    dest_dir.mkdir(parents=True, exist_ok=True)
                    dest_path = dest_dir / img_path.name
                    shutil.copy(str(img_path), str(dest_path))
                    update_image_exif(dest_path, label)

                    writer.writerow({
                        "SourceType": "Image", "PredictedPlace": label, "Confidence": f"{confidence:.4f}",
                        "OriginalPath": str(img_path.resolve()), "OutputPath": str(dest_path.resolve()),
                        "IsVideo": 0, "Sharpness": "N/A"
                    })
                    mark_processed(key)
                else:
                    label = class_names[pred_index]
                    logging.info(f"Immagine: {img_path.name} -> Classe: {label} (Conf: {confidence:.3f}) -> RIFIUTATA (soglia non raggiunta)")

        # BLOCCO DI INFERENZA SUI VIDEO
        logging.info(f"Inizio Inferenza su Video in: {args.vid_dir} ")
        vid_dir = Path(args.vid_dir)
        if vid_dir.is_dir():
            video_files = sorted(list(vid_dir.glob("*.mp4")) + list(vid_dir.glob("*.mov")) + list(vid_dir.glob("*.avi")))
            logging.info(f"Trovati {len(video_files)} video da processare.")

            for vid_path in tqdm(video_files, desc="Processing Videos"):
                key = f"VID::{vid_path.resolve()}"
                if is_already_processed(key):
                    logging.info(f"Video già processato, saltato: {vid_path.name}")
                    continue

                temp_video_frame_dir = TEMP_FRAME_EXTRACT_DIR / vid_path.stem
                frame_paths = extract_frames(vid_path, temp_video_frame_dir, args.frame_interval)
                if not frame_paths: continue

                valid_frames_data = [d for d in [load_and_preprocess_img_infer(str(p)) for p in frame_paths] if d is not None]
                if not valid_frames_data:
                    shutil.rmtree(temp_video_frame_dir, ignore_errors=True)
                    continue

                predictions = model.predict(tf.stack(valid_frames_data), batch_size=BATCH_SIZE)
                pred_indices = np.argmax(predictions, axis=1)
                pred_confidences = np.max(predictions, axis=1)

                # 1. Conta le occorrenze per ogni classe
                counts = np.bincount(pred_indices, minlength=len(class_names))
                total_valid_frames = len(pred_indices)

                # 2. Definisci una soglia di presenza (es. 10% dei frame) per considerare una scena significativa
                presence_threshold_percentage = 0.10

                # 3. Identifica gli indici di tutte le classi che superano la soglia
                significant_class_indices = [
                    idx for idx, count in enumerate(counts)
                    if (count / total_valid_frames) >= presence_threshold_percentage
                ]

                # Se nessuna classe supera la soglia, come fallback si può prendere quella più votata
                if not significant_class_indices:
                    logging.warning(f"Nessuna classe ha superato la soglia di presenza per {vid_path.name}. Uso la classe più votata come fallback.")
                    significant_class_indices = [np.argmax(counts)]

                # 4. Inizializza le liste per raccogliere i risultati aggregati del video
                final_video_labels = []
                all_best_frames_paths = []
                confidence_scores = []
                sharpness_scores = []

                # 5. Itera su OGNI classe significativa trovata e seleziona i frame migliori per ciascuna
                for class_idx in significant_class_indices:
                    video_label = class_names[class_idx]
                    final_video_labels.append(video_label)

                    # Calcola la confidenza media SOLO per i frame appartenenti a QUESTA classe
                    class_confidence = float(np.mean(pred_confidences[pred_indices == class_idx]))
                    confidence_scores.append(f"{video_label}:{class_confidence:.3f}")
                    logging.info(f"Video: {vid_path.name} -> Rilevata classe significativa: {video_label} (Presenza: {counts[class_idx]/total_valid_frames:.1%}, Conf. media: {class_confidence:.3f})")

                    # Seleziona i frame migliori PER QUESTA CLASSE
                    candidate_frames_info = []
                    for i, frame_path in enumerate(frame_paths):
                        if pred_indices[i] == class_idx:
                            sharpness = calculate_sharpness(cv2.imread(str(frame_path)))
                            if sharpness >= args.sharpness_threshold:
                                candidate_frames_info.append((sharpness, frame_path))

                    candidate_frames_info.sort(key=lambda x: x[0], reverse=True)
                    best_frames_to_save = candidate_frames_info[:args.frames_per_place_per_video]

                    if best_frames_to_save:
                        video_best_frames_dir = BEST_FRAMES_OUTPUT_DIR / video_label / vid_path.stem
                        video_best_frames_dir.mkdir(parents=True, exist_ok=True)
                        sharpness_scores.append(f"{video_label}:{best_frames_to_save[0][0]:.1f}")

                        for sharpness, frame_p in best_frames_to_save:
                            out_filename = f"{vid_path.stem}__{video_label}__frame{frame_p.stem.split('frame')[-1]}_sharp{sharpness:.0f}.jpg"
                            out_path = video_best_frames_dir / out_filename
                            shutil.copy(str(frame_p), str(out_path))
                            update_image_exif(out_path, video_label)
                            all_best_frames_paths.append(str(out_path.resolve()))

                # 6. Scrive i risultati aggregati del video (con tutte le classi) nel file CSV.
                writer.writerow({
                    "SourceType": "Video",
                    "PredictedPlace": ",".join(final_video_labels),
                    "Confidence": ",".join(confidence_scores),
                    "OriginalPath": str(vid_path.resolve()),
                    "OutputPath": ";".join(all_best_frames_paths),
                    "IsVideo": 1,
                    "Sharpness": ",".join(sharpness_scores)
                })

                mark_processed(key)
                shutil.rmtree(temp_video_frame_dir, ignore_errors=True)
        else:
            logging.warning(f"La directory dei video {vid_dir} non esiste.")

    logging.info("Processo di Inferenza Concluso")

### **Esecuzione dell'Inferenza**

Questa cella avvia il processo di inferenza. Il suo scopo è configurare tutti i parametri necessari, avviare l'analisi e gestire eventuali errori in modo controllato, fornendo un feedback chiaro all'utente.

#### **1. Pulizia dello Stato di Avanzamento (Reset)**

```python
if log_file_path.exists():
    log_file_path.unlink()
```
All'inizio della cella, viene deliberatamente cancellato il file `processed.log`, se esiste.
*  Garantisce che, ogni volta che si esegue questa cella, lo script analizzerà di nuovo **tutti** i file presenti nelle directory di input, anche se erano già stati processati in una sessione precedente. Questo è utile per testare modifiche o per assicurarsi di avere un set di risultati completo e aggiornato.
*   **Alternativa**: Se si volesse analizzare solo i file nuovi aggiunti tra un'esecuzione e l'altra, basterebbe commentare o rimuovere questo blocco di pulizia. Il sistema diventerebbe così **incrementale**, risparmiando tempo su grandi dataset.

#### **2. Configurazione dei Parametri (`args_infer_config`)**

Raccoglie tutti i parametri di configurazione in un unico oggetto per passarlo in modo pulito e ordinato alla funzione `infer`.
* Viene utilizzato `SimpleNamespace`, per creare un oggetto "contenitore". Invece di passare dieci argomenti separati alla funzione `infer`, le passiamo solo `args_infer_config`. All'interno della funzione, si accederà ai valori con una notazione chiara come `args.img_dir` o `args.sharpness_threshold`.
*   **Parametri Inclusi**:
    *   **Percorsi di Input/Output**: Dove trovare le immagini/video e dove salvare i risultati.
    *   **Iperparametri di Inferenza**: Parametri specifici per l'analisi, come l'intervallo di estrazione dei frame (`FRAME_INTERVAL`) o la soglia di nitidezza (`SHARPNESS_THRESHOLD`).

In [ ]:
# Blocco di Pulizia del Log di Processamento
# A differenza del blocco di pulizia nella cella del training, questo è attivo di default.
# Il suo scopo è cancellare il file `processed.log` prima di ogni esecuzione.
# Questo assicura che l'inferenza venga eseguita su TUTTI i file presenti nelle
# cartelle di input, anche se sono già stati analizzati in una sessione precedente.
# Se volessi un comportamento "incrementale" (analizzare solo i file nuovi),
# dovresti commentare o rimuovere questo blocco.

# Definisce il percorso del file di log.
log_file_path = DEFAULT_OUTPUT / "processed.log"
# Controlla se il file esiste.
if log_file_path.exists():
    # Stampa messaggi informativi per l'utente.
    print(f"Cancellazione del vecchio file di log: {log_file_path}")
    # Cancella il file.
    log_file_path.unlink()
    print("Log cancellato. Tutti i file verranno ri-processati.")


# Configurazione per l'Inferenza
# Crea un oggetto `SimpleNamespace` per contenere tutti i parametri da passare
# alla funzione `infer`. Questo raggruppa ordinatamente la configurazione.
args_infer_config = SimpleNamespace(
    # Percorsi delle directory di input per immagini e video.
    img_dir=str(DEFAULT_IMG_DIR),
    vid_dir=str(DEFAULT_VID_DIR),

    # Percorsi di output.
    output=DEFAULT_OUTPUT,
    csv_path=DEFAULT_CSV,

    # Parametri specifici per il processamento dei video.
    frame_interval=FRAME_INTERVAL,
    sharpness_threshold=SHARPNESS_THRESHOLD,
    frames_per_place_per_video=FRAMES_PER_PLACE_PER_VIDEO,
)

# Stampa di Verifica dei Parametri
# Stampa un riepilogo delle configurazioni chiave per l'inferenza.
# È una buona pratica per avere una conferma visiva che lo script
# sta leggendo e scriverà nelle cartelle corrette.
print("\n--- Parametri per l'Inferenza ---")
print(f"Directory Immagini Input: {args_infer_config.img_dir}")
print(f"Directory Video Input: {args_infer_config.vid_dir}")
print(f"Directory Output: {args_infer_config.output}")
print(f"File CSV Risultati: {args_infer_config.csv_path}")

# ESECUZIONE DELL'INFERENZA
try:
    # Chiama la funzione `infer` con la configurazione appena creata.
    # A questo punto, lo script inizierà a caricare il modello e a processare
    # i file uno per uno, come definito nella cella 10.
    infer(args_infer_config)

    # Se la funzione `infer` termina senza errori, stampa un messaggio di successo
    # e un riepilogo di dove trovare i risultati. Questo è molto utile per l'utente.
    print(f"\n--- INFERENZA COMPLETATA ---")
    print(f"Controlla i risultati nel file CSV: {args_infer_config.csv_path}")
    print(f"Le immagini classificate si trovano in: {args_infer_config.output / 'classified_images'}")
    print(f"I frame migliori dei video si trovano in: {BEST_FRAMES_OUTPUT_DIR}")

# Cattura qualsiasi errore imprevisto durante l'inferenza.
except Exception as e:
    # Stampa un messaggio di errore e la traccia completa dello stack per facilitare il debug.
    print(f"ERRORE INASPETTATO DURANTE L'INFERENZA: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# NUOVA CELLA - DEFINIZIONE DELLA FUNZIONE DI TEST

def test(args):
    """Esegue la valutazione del modello su un set di dati di test separato."""
    setup_logging(args.output)
    logging.info("--- Inizio Processo di Test ---")

    model_path = args.output / "place_model.keras"
    if not model_path.exists():
        logging.error(f"Modello non trovato in: {model_path}. Esegui prima il training.")
        return

    try:
        model = models.load_model(model_path)
        logging.info(f"Modello caricato da: {model_path}")
    except Exception as e:
        logging.error(f"Errore caricamento modello: {e}")
        return

    test_dir_path = Path(args.test_dir)
    if not test_dir_path.is_dir():
        logging.error(f"Directory di test non trovata: {test_dir_path}")
        return

    test_ds, class_names = make_dataset(test_dir_path, shuffle=False, augment=False)
    if not test_ds:
        logging.error("Creazione del dataset di test fallita.")
        return

    logging.info(f"Dataset di test caricato con classi: {class_names}")
    logging.info("Valutazione del modello sul set di test in corso...")

    results = model.evaluate(test_ds, verbose=1)

    logging.info("--- Risultati del Test ---")
    print("\n--- Risultati del Test ---")
    print(f"Loss sul set di test: {results[0]:.4f}")
    print(f"Accuracy sul set di test: {results[1] * 100:.2f}%")
    logging.info(f"Loss: {results[0]:.4f} - Accuracy: {results[1] * 100:.2f}%")
    logging.info("--- Processo di Test Concluso ---")

print("Funzione di Test definita.")

In [ ]:
# NUOVA CELLA - ESECUZIONE DEL TEST - DA CONTROLLARE

args_test_config = SimpleNamespace(
    test_dir=str(DEFAULT_TEST_DIR),
    output=DEFAULT_OUTPUT
)

try:
    print("\nLancio della valutazione sul set di test...")
    test(args_test_config)
except Exception as e:
    print(f"ERRORE INASPETTATO DURANTE IL TEST: {e}")
    import traceback
    traceback.print_exc()